<a href="https://colab.research.google.com/github/MarciaFG/skill-flow/blob/main/Flows_1980_2022_first_level_for.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Academic Mobility Flows**

Author: Marcia R. Ferreira (Complexity Science Hub Vienna & TU Wien)

Date: September 28, 2022

Input: Dimensions database on BigQuery

Output: GBQ Dimensions.ai



## Initialization

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Wed Mar 29 07:38:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Install Drivers

In [2]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
#!pip install psutil
#!pip install humanize
#!pip install pynput
#pip install plotly==5.4.0
# libraries
import psutil
import humanize
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import requests
import torch
import nltk
import GPUtil as GPU
import plotly.graph_objs as go
import plotly.io as pio

# plotting
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

from google.cloud import bigquery
from google.colab import files
%load_ext google.colab.data_table
%load_ext google.cloud.bigquery

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7408 sha256=9299010db5f5fd091e8e9da4534ce97ec6b418e4ddcdc3cda2b68a476ac5f345
  Stored in directory: /root/.cache/pip/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil


In [3]:
# only one GPU on Colab and isn’t guaranteed
import psutil
import os
import humanize
import GPUtil as GPU

GPUs = GPU.getGPUs()
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ),\
       " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB"\
       .format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 87.7 GB  | Proc size: 455.6 MB
GPU RAM Free: 40513MB | Used: 0MB | Util   0% | Total 40960MB


**Loading data from Google Drive (If needed)**

In [ ]:
# run this to upload files
# from google.colab import files
# uploaded = files.upload() 

**Mounting the Google Drive folder**

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# let's test it
#with open('/content/drive/My Drive/foo.txt', 'w') as f:
#  f.write('Hello Google Drive!')
#!cat /content/drive/My\ Drive/foo.txt

Mounted at /content/drive
Hello Google Drive!

**Runtime credentials**

In [4]:
# Provide your credentials to the runtime
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


**Declare the Cloud project ID which will be used throughout this notebook**

In [5]:
# declare your project 
project_id = "cshdimensionstest"

# **Institutional Mobility**

# PART I - Preprocessing

## **1980-2022**
- Filtering by authors who have at least 2 publications 
- Filtering by authors whose first publication year is at least 1980
- Filtering by authors who have published between 1980 and 2022

## (1) Load Data from GBQ

**NOT USE**

In [ ]:
%%bigquery --project $project_id

# first we collect all researchers who have published after 1980
CREATE OR REPLACE TABLE cshdimensionstest.test.authors_after1980 AS
SELECT
  id AS researcher_ids,
  redirect as new_id,
  first_publication_year,
  research_orgs AS org_list
FROM
  `dimensions-ai.data_analytics.researchers`
  , UNNEST(redirect)
  WHERE first_publication_year >= 1980
  AND research_orgs IS NOT NULL;

**Oops there seems to be an issue with the researchers getting new ids and being split over time. This means that we should not use this table to select the researchers.**

# Start

In [124]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022 AS

WITH researcher_first_pub_year AS (
  SELECT
    researcher_id,
    MIN(p.year) AS first_pub_year
  FROM
    `dimensions-ai.data_analytics.publications` p,
    UNNEST(authors) AS researchers
  GROUP BY
    researcher_id
  HAVING MIN(p.year) >=1980
),
unnested_grid_ids AS (
SELECT
  researchers.researcher_id,
  grid as grid_ids,
  p.id AS pub_id,
  p.year,
  category.id
FROM
  `dimensions-ai.data_analytics.publications` p,
  UNNEST(authors) AS researchers,
  UNNEST(researchers.grid_ids) AS grid,
  UNNEST(category_for.first_level.full) AS category
  JOIN
    researcher_first_pub_year rp
  ON
    rp.researcher_id = researchers.researcher_id
  WHERE
    category.id IS NOT NULL                     -- only publications that have a category id
    AND researchers.researcher_id IS NOT NULL
    AND researchers.grid_ids IS NOT NULL
    AND p.year BETWEEN 1980 AND 2022            -- only publications between 1980-2022
),
grid_id_count AS (
  SELECT
    researcher_id,
    COUNT(DISTINCT grid_ids) AS grid_id_count
  FROM
    unnested_grid_ids 
  GROUP BY
    researcher_id
)
SELECT
  u.researcher_id as researcher_ids,
  u.pub_id,
  u.year,
  grid_id_count,
  grid_ids,
  u.id
FROM
  unnested_grid_ids u
JOIN
  grid_id_count g
ON
  u.researcher_id = g.researcher_id
WHERE
  g.grid_id_count >= 2                          -- we take researchers who have had at least 2 affiliations either simulateously or not
ORDER BY
  u.pub_id,
  u.year;

Query is running:   0%|          |

""


In [132]:
%%bigquery --project $project_id
select * 
from cshdimensionstest.test.au_pub_history_1980_2022
order by researcher_ids, year, pub_id, grid_ids
limit 50; 

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,pub_id,year,grid_id_count,grid_ids,id
0,ur.01000000010.53,pub.1077606951,2007,2,grid.461843.c,80003
1,ur.01000000010.53,pub.1028748827,2011,2,grid.461843.c,80003
2,ur.01000000010.53,pub.1040612737,2012,2,grid.461843.c,80003
3,ur.01000000010.53,pub.1004493655,2013,2,grid.461843.c,80003
4,ur.01000000010.53,pub.1039762887,2013,2,grid.461843.c,80003
5,ur.01000000010.53,pub.1040126984,2013,2,grid.461843.c,80003
6,ur.01000000010.53,pub.1049771699,2013,2,grid.461843.c,80003
7,ur.01000000010.53,pub.1078832884,2013,2,grid.461843.c,80003
8,ur.01000000010.53,pub.1121667807,2013,2,grid.461843.c,80003
9,ur.01000000010.53,pub.1121815093,2013,2,grid.461843.c,80003


In [ ]:
%%bigquery --project $project_id
SELECT COUNT(DISTINCT researcher_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022; --6,435,674
SELECT COUNT(DISTINCT grid_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022; --69,247
SELECT COUNT(DISTINCT pub_id) FROM cshdimensionstest.test.au_pub_history_1980_2022; --44,820,788

#### *(1.1) Population Statistics*

In [ ]:
%%bigquery --project $project_id 


CREATE OR REPLACE TABLE cshdimensionstest.test.grid_population_1980_2022 AS

WITH researcher_first_pub_year AS (
  SELECT
    researcher_id,
    MIN(p.year) AS first_pub_year
  FROM
    `dimensions-ai.data_analytics.publications` p,
    UNNEST(authors) AS researchers
  GROUP BY
    researcher_id
  HAVING MIN(p.year) >=1980
),
unnested_grid_ids AS (
SELECT
  researchers.researcher_id,
  grid as grid_ids,
  p.id AS pub_id,
  p.year,
  category.id
FROM
  `dimensions-ai.data_analytics.publications` p,
  UNNEST(authors) AS researchers,
  UNNEST(researchers.grid_ids) AS grid,
  UNNEST(category_for.first_level.full) AS category
  JOIN
    researcher_first_pub_year rp
  ON
    rp.researcher_id = researchers.researcher_id
  WHERE
    category.id IS NOT NULL                     -- only publications that have a category id
    AND researchers.researcher_id IS NOT NULL
    AND researchers.grid_ids IS NOT NULL
    AND p.year BETWEEN 1980 AND 2022            -- only publications between 1980-2022
),
researchers_per_year_per_institution AS (
  SELECT
    grid_ids,
    year,
    COUNT(DISTINCT researcher_id) AS total_researchers
  FROM
    unnested_grid_ids
  GROUP BY
    grid_ids,
    year
)
SELECT *
FROM researchers_per_year_per_institution;

SELECT a.*, b.name
FROM cshdimensionstest.test.grid_population_1980_2022 a
JOIN dimensions-ai.data_analytics.grid b on a.grid_ids=b.id
ORDER BY  grid_ids, year
LIMIT 1000;

Executing query with job ID: 7fe8ef26-65bc-4c52-ae48-85272d37c752
Query executing: 17.25s

In [ ]:
-- the number of distinct researchers per year per institution (grid):
CREATE OR REPLACE TABLE cshdimensionstest.test.grid_population_1980_2022 AS
WITH unnested_grids AS (
  SELECT
    a.researcher_ids,
    TO_JSON_STRING(org_list) AS org_list,
    p.id AS pub_id,
    p.year,
    category.id,
    grid_ids
  FROM
    `dimensions-ai.data_analytics.publications` p,
    UNNEST(authors) AS researchers,
    UNNEST(grid_ids) AS grid_ids,
    UNNEST(category_for.first_level.full) AS category
  JOIN
    cshdimensionstest.test.authors_after1980 a
  ON
    a.researcher_ids = researchers.researcher_id
  WHERE
    category.id IS NOT NULL
    AND p.year BETWEEN 1980 AND 2022
),
researchers_per_year_per_institution AS (
  SELECT
    grid_ids,
    year,
    COUNT(DISTINCT researcher_ids) AS total_researchers
  FROM
    unnested_grids
  GROUP BY
    grid_ids,
    year
)
SELECT *
FROM researchers_per_year_per_institution;

SELECT *
FROM cshdimensionstest.test.grid_population_1980_2022
ORDER BY
  grid_ids,
  year
LIMIT
  20;


In [ ]:
%%bigquery --project $project_id

-- Count non-mobile researchers at the institutional level per year
CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022_non_mobile_org_counts AS
WITH unnested_grid_ids AS (
  SELECT
    a.researcher_ids,
    TO_JSON_STRING(org_list) AS org_list,
    p.id AS pub_id,
    p.year,
    category.id,
    grid_ids
  FROM
    `dimensions-ai.data_analytics.publications` p,
    UNNEST(authors) AS researchers,
    UNNEST(grid_ids) AS grid_ids,
    UNNEST(category_for.first_level.full) AS category
  JOIN
    cshdimensionstest.test.authors_after1980 a
  ON
    a.researcher_ids = researchers.researcher_id
  WHERE
    category.id IS NOT NULL
    AND p.year BETWEEN 1980 AND 2022
),
grid_id_count AS (
  SELECT
    researcher_ids,
    org_list,
    COUNT(DISTINCT grid_ids) AS grid_id_count
  FROM
    unnested_grid_ids 
  GROUP BY
    researcher_ids,
    org_list
)
SELECT
  u.researcher_ids,
  u.pub_id,
  u.year,
  u.id,
  u.grid_ids,
  g.grid_id_count
FROM
  unnested_grid_ids u
JOIN
  grid_id_count g
ON
  u.researcher_ids = g.researcher_ids
  AND u.org_list = g.org_list
WHERE
  g.grid_id_count = 1 -- we take only non-mobile researchers
ORDER BY
  u.pub_id,
  u.year;

SELECT *
FROM cshdimensionstest.test.au_pub_history_1980_2022_non_mobile_org_counts
ORDER BY
  researcher_ids
LIMIT
  20;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,pub_id,year,id,grid_ids,grid_id_count
0,ur.010000000127.95,pub.1052932477,2012,80012,grid.452405.2,1
1,ur.010000000127.95,pub.1030316458,2010,80012,grid.452405.2,1
2,ur.010000000127.95,pub.1012465979,2009,80001,grid.452405.2,1
3,ur.010000000127.95,pub.1014384761,2006,80012,grid.452405.2,1
4,ur.010000000127.95,pub.1013572221,2012,80012,grid.452405.2,1
5,ur.010000000127.95,pub.1019392859,2007,80012,grid.452405.2,1
6,ur.010000000127.95,pub.1012465979,2009,80012,grid.452405.2,1
7,ur.01000000017.89,pub.1002418025,2008,80011,grid.415193.b,1
8,ur.01000000017.89,pub.1009202825,2006,80022,grid.415193.b,1
9,ur.01000000017.89,pub.1002418025,2008,80022,grid.415193.b,1


In [ ]:
%%bigquery --project $project_id 

CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022_non_mobile_researchers AS
SELECT 
  a.grid_ids,
  a.year,
  total_researchers,
  COUNT(DISTINCT b.researcher_ids) AS non_mobile_researchers
FROM 
  cshdimensionstest.test.grid_population_1980_2022 a
JOIN 
  cshdimensionstest.test.au_pub_history_1980_2022_non_mobile_org_counts b 
ON 
  a.grid_ids=b.grid_ids 
  AND a.year=b.year
GROUP BY 
  grid_ids, 
  year, 
  total_researchers;

CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022_population_counts AS
SELECT 
  a.grid_ids,
  a.year,
  total_researchers,
  non_mobile_researchers,
  COUNT(DISTINCT b.researcher_ids) AS mobile_researchers
FROM 
  cshdimensionstest.test.au_pub_history_1980_2022_non_mobile_researchers a
JOIN 
  cshdimensionstest.test.au_pub_history_1980_2022 b 
ON 
  a.grid_ids=b.grid_ids 
  and a.year=b.year
GROUP BY 
  grid_ids, 
  year, 
  total_researchers, 
  non_mobile_researchers;


CREATE OR REPLACE TABLE cshdimensionstest.test.au_pub_history_1980_2022_population_statistics AS 
SELECT 
  *, 
  ROUND((non_mobile_researchers / total_researchers) * 100, 3) AS pct_non_mobile,
  ROUND((mobile_researchers / total_researchers) * 100, 3) AS pct_mobile
FROM 
  cshdimensionstest.test.au_pub_history_1980_2022_population_counts;


SELECT 
  * 
FROM 
  cshdimensionstest.test.au_pub_history_1980_2022_population_statistics
ORDER BY 
  grid_ids, 
  year
LIMIT 
  50;


Query is running:   0%|          |

Downloading:   0%|          |

,grid_ids,year,total_researchers,non_mobile_researchers,mobile_researchers,pct_non_mobile,pct_mobile
0,grid.1001.0,1980,138,96,42,69.565,30.435
1,grid.1001.0,1981,179,121,58,67.598,32.402
2,grid.1001.0,1982,241,149,92,61.826,38.174
3,grid.1001.0,1983,252,137,115,54.365,45.635
4,grid.1001.0,1984,338,191,147,56.509,43.491
5,grid.1001.0,1985,380,190,190,50.000,50.000
6,grid.1001.0,1986,386,176,210,45.596,54.404
7,grid.1001.0,1987,466,205,261,43.991,56.009
8,grid.1001.0,1988,566,236,330,41.696,58.304
9,grid.1001.0,1989,506,208,298,41.107,58.893


In [ ]:
%%bigquery --project $project_id
--SELECT COUNT(distinct researcher_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022; -- 6,427,173 distinct researchers
--SELECT COUNT(distinct grid_ids) FROM cshdimensionstest.test.au_pub_history_1980_2022; -- 69,228 unique organizations
--SELECT COUNT(distinct pub_id) FROM cshdimensionstest.test.au_pub_history_1980_2022; -- 44,781,885 publications of any document type

## (2) Trajectories

**Let's check the table**


*   ur.013012771111.87
*   Rodrigo	Costas



In [ ]:
%%bigquery --project $project_id 

 create or replace table cshdimensionstest.test.sequence_1980_2022 as 
  select distinct researcher_ids, 
    year, 
    dense_rank() over (
      partition by researcher_ids 
      order by 
        year asc
    ) as t 
  from `cshdimensionstest.test.au_pub_history_1980_2022`
  order by 
    researcher_ids, 
    year, 
  t;

create or replace table cshdimensionstest.test.pub_trajectory_1980_2022 as
select a.*, b.t
from cshdimensionstest.test.au_pub_history_1980_2022 a
join cshdimensionstest.test.sequence_1980_2022 b
on a.researcher_ids=b.researcher_ids and a.year=b.year;

CREATE OR REPLACE TABLE cshdimensionstest.test.institutional_trajectory_1980_2022 AS
SELECT DISTINCT researcher_ids, year, grid_ids, t 
FROM cshdimensionstest.test.pub_trajectory_1980_2022;

SELECT * FROM cshdimensionstest.test.pub_trajectory_1980_2022
where researcher_ids = 'ur.013012771111.87' 
order by t, pub_id
limit 5;

In [ ]:
%%bigquery --project $project_id 
 create or replace table cshdimensionstest.test.simple_sequence_1980_2022 as 
select distinct 
	  researcher_ids
	, grid_ids
  , count(Distinct pub_id) as t_pubs
	, min(year) as start_year
	, max(Year) as end_year
	, dense_rank() over 
(
    partition by researcher_ids 
    order by 
    min(year), max(year) asc
) as t
from `cshdimensionstest.test.au_pub_history_1980_2022`
group by researcher_ids, grid_ids;

SELECT * FROM cshdimensionstest.test.simple_sequence_1980_2022
where researcher_ids = 'ur.013012771111.87' 
order by t;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,grid_ids,t_pubs,start_year,end_year,t
0,ur.013012771111.87,grid.4711.3,3,2007,2010,1
1,ur.013012771111.87,grid.5132.5,81,2009,2022,2
2,ur.013012771111.87,grid.11956.3a,34,2017,2022,3


In [20]:
%%bigquery --project $project_id 
SELECT * FROM cshdimensionstest.test.sequence_1980_2022 
where researcher_ids = 'ur.010022351501.26' 
--order by t;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,year,t
0,ur.010022351501.26,2019,1
1,ur.010022351501.26,2022,2


In [21]:
%%bigquery --project $project_id 
SELECT * FROM cshdimensionstest.test.simple_sequence_1980_2022 
where researcher_ids = 'ur.010022351501.26' 
--order by t;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,grid_ids,t_pubs,start_year,end_year,t
0,ur.010022351501.26,grid.410726.6,1,2019,2019,1
1,ur.010022351501.26,grid.450302.0,1,2019,2019,1
2,ur.010022351501.26,grid.11135.37,2,2022,2022,2


In [22]:
%%bigquery --project $project_id 
SELECT * FROM cshdimensionstest.test.institutional_trajectory_1980_2022 
where researcher_ids = 'ur.010022351501.26' 
--order by t;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,year,grid_ids,t
0,ur.010022351501.26,2019,grid.450302.0,1
1,ur.010022351501.26,2019,grid.410726.6,1
2,ur.010022351501.26,2022,grid.11135.37,2


# PART II - Mobility Network

## (3) Mobility Flows

We will split the calculation of the network flows:
1. Institutions
2. Cities
3. Countries
4. FOR 2-digit field
- This script is at the institutional level
---



We can later on think of costumisable layers such as NUTS2 etc

### ***(3.1) Author Institutional Flows***
In this part we focus on flows at the level of institutions

In [ ]:
%%bigquery --project $project_id 

CREATE TABLE cshdimensionstest.test.simple_sequence_1980_2022_clustered 
(
  -- Add your table schema here, e.g.
  researcher_ids STRING,
  grid_ids STRING,
  start_year INT64,
  t INT64
)
CLUSTER BY researcher_ids
OPTIONS(
  description="A clustered table of simple_sequence_1980_2022"
)
AS
SELECT researcher_ids, grid_ids, start_year, t FROM cshdimensionstest.test.simple_sequence_1980_2022;

SELECT * FROM cshdimensionstest.test.simple_sequence_1980_2022_clustered 
where researcher_ids = 'ur.013012771111.87' 
order by t;

   ATT: This is the most computationally expensive table, becareful with running it too many times


In [ ]:
%%bigquery --project $project_id 
# now we have everything we need to construct the flows at the institutional level
create or replace table cshdimensionstest.test.flows_1980_2022 as 
  select 
    a.researcher_ids,
    a.grid_ids as unit1,
    b.grid_ids as unit2,
    a.start_year as p1,
    b.start_year as p2
  from
        cshdimensionstest.test.simple_sequence_1980_2022_clustered a 
    inner join
        cshdimensionstest.test.simple_sequence_1980_2022_clustered b 
        on a.researcher_ids = b.researcher_ids 
  where a.t = b.t - 1;

# check the table
select * 
from cshdimensionstest.test.flows_1980_2022
order by researcher_ids, p1, p2, unit1, unit2 
limit 50;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2
0,ur.01000000010.53,grid.461843.c,grid.506261.6,2007,2015
1,ur.010000000201.99,grid.260542.7,grid.412046.5,2014,2014
2,ur.010000000201.99,grid.453140.7,grid.412046.5,2014,2014
3,ur.010000000201.99,grid.412046.5,grid.39158.36,2014,2020
4,ur.01000000021.08,grid.136593.b,grid.17091.3e,2010,2013
5,ur.01000000021.08,grid.17091.3e,grid.469958.f,2013,2016
6,ur.01000000021.08,grid.469958.f,grid.10251.37,2016,2021
7,ur.01000000021.08,grid.469958.f,grid.440269.d,2016,2021
8,ur.010000000667.71,grid.412795.c,grid.419886.a,2020,2022
9,ur.010000001234.00,grid.412370.3,grid.414474.6,2020,2021


 ### ***(3.2) Cross-Institutional Flows***
 
 Are the flows exchanged between two instituions at a given calendar year

In [ ]:
%%bigquery --project $project_id

-- Calculate the total flows between institutional pairs
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutional_flows AS
SELECT 
  unit1 AS geoid_o,
  unit2 AS geoid_d,
  p2 AS date_d,
  COUNT(DISTINCT researcher_ids) AS total_flows, # author flows
FROM cshdimensionstest.test.flows_1980_2022
GROUP BY 
  geoid_o, 
  geoid_d, 
  date_d;

-- Check the table 
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_institutional_flows 
ORDER BY 
  geoid_o, 
  geoid_d, 
  date_d 
LIMIT 50;


### ***(3.3) Total Flows by Institution***

***Overall flows by institution over time***
- outgoing flows (counts and percentage)
- incoming flows (counts and percentage)
- total flows (counts)
- total net flows (counts)
- net_mobility_rate


In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutional_total_flows_agg AS
SELECT
  node,
  date_d,
  SUM(outgoing_flows) AS outgoing_flows,
  ROUND(SUM(outgoing_flows) / (SUM(outgoing_flows) + SUM(incoming_flows)) * 100, 1) AS percentage_outflows,
  SUM(incoming_flows) AS incoming_flows,
  ROUND(SUM(incoming_flows) / (SUM(outgoing_flows) + SUM(incoming_flows)) * 100, 1) AS percentage_inflows,
  SUM(outgoing_flows) + SUM(incoming_flows) AS total_flows,
  SUM(incoming_flows) - SUM(outgoing_flows) AS total_net_flows,
  ROUND((SUM(incoming_flows) - SUM(outgoing_flows)) / (SUM(incoming_flows) + SUM(outgoing_flows)) * 100, 1) AS net_mobility_rate
FROM (
  SELECT
    unit1 AS node,
    p2 as date_d,
    COUNT(DISTINCT researcher_ids) AS outgoing_flows,
    0 AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2022
  GROUP BY
    unit1, p2

  UNION ALL

  SELECT
    unit2 AS node,
    p2 as date_d,
    0 AS outgoing_flows,
    COUNT(DISTINCT researcher_ids) AS incoming_flows
  FROM
    cshdimensionstest.test.flows_1980_2022
  GROUP BY
    unit2, p2
) 
AS flows
GROUP BY
  node, date_d;

-- Check the table 
SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_institutional_total_flows_agg
ORDER BY 
  node, 
  date_d 
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,percentage_outflows,incoming_flows,percentage_inflows,total_flows,total_net_flows,net_mobility_rate
0,grid.1001.0,1980,5,55.6,4,44.4,9,-1,-11.1
1,grid.1001.0,1981,5,41.7,7,58.3,12,2,16.7
2,grid.1001.0,1982,9,34.6,17,65.4,26,8,30.8
3,grid.1001.0,1983,15,41.7,21,58.3,36,6,16.7
4,grid.1001.0,1984,23,57.5,17,42.5,40,-6,-15.0
5,grid.1001.0,1985,27,39.7,41,60.3,68,14,20.6
6,grid.1001.0,1986,43,65.2,23,34.8,66,-20,-30.3
7,grid.1001.0,1987,44,44.4,55,55.6,99,11,11.1
8,grid.1001.0,1988,72,55.0,59,45.0,131,-13,-9.9
9,grid.1001.0,1989,77,59.2,53,40.8,130,-24,-18.5


## (4) Indicators

### ***(4.1) Academic Age***
The average academic age of inflowing and outflowing researchers for each node and year by joining the two subsets of data on the researcher ID

- case:ur.013012771111.87
- Rodrigo Costas
- grid.5132.5 

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age AS

WITH academic_age AS (
  SELECT 
    a.researcher_ids,
    b.unit1,
    b.unit2,
    a.min_academic_age,
    b.last_year_at_unit1,
    (b.last_year_at_unit1 - a.min_academic_age + 1) AS outgoing_academic_age,
    (b.last_year_at_unit2 - a.min_academic_age + 1) AS incoming_academic_age
  FROM (
    SELECT researcher_ids, MIN(p1) AS min_academic_age
    FROM cshdimensionstest.test.flows_1980_2022
    GROUP BY researcher_ids
  ) AS a
  JOIN (
    SELECT 
      researcher_ids, 
      unit1, 
      unit2,
      MAX(p2) AS last_year_at_unit1,
      MAX(p2) AS last_year_at_unit2
    FROM cshdimensionstest.test.flows_1980_2022
    GROUP BY researcher_ids, unit1, unit2
  ) AS b
  ON a.researcher_ids = b.researcher_ids
)
SELECT * 
FROM academic_age;

In [ ]:
%%bigquery --project $project_id
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_1 AS
SELECT a.*, b.outgoing_academic_age, b.incoming_academic_age 
FROM cshdimensionstest.test.flows_1980_2022 a
JOIN cshdimensionstest.test.flows_1980_2022_age b 
  ON a.researcher_ids=b.researcher_ids 
  AND a.unit1=b.unit1 
  AND a.unit2=b.unit2;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_age_1
ORDER BY researcher_ids, p1, p1, unit1, unit2
LIMIT 10;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,outgoing_academic_age,incoming_academic_age
0,ur.01000000010.53,grid.461843.c,grid.506261.6,2007,2015,9,9
1,ur.010000000201.99,grid.260542.7,grid.412046.5,2014,2014,1,1
2,ur.010000000201.99,grid.412046.5,grid.39158.36,2014,2020,7,7
3,ur.010000000201.99,grid.453140.7,grid.412046.5,2014,2014,1,1
4,ur.01000000021.08,grid.136593.b,grid.17091.3e,2010,2013,4,4
5,ur.01000000021.08,grid.17091.3e,grid.469958.f,2013,2016,7,7
6,ur.01000000021.08,grid.469958.f,grid.10251.37,2016,2021,12,12
7,ur.01000000021.08,grid.469958.f,grid.440269.d,2016,2021,12,12
8,ur.010000000667.71,grid.412795.c,grid.419886.a,2020,2022,3,3
9,ur.010000001234.00,grid.412370.3,grid.414474.6,2020,2021,2,2




*   We need to count the researchers moving based on their seniority (i.e., academic age)


In [ ]:
%%bigquery --project $project_id
--SELECT AVG(incoming_academic_age) from cshdimensionstest.test.flows_1980_2022_age_1 
-- mean = 9.6082983842395
--SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_age_1 
-- rows = 17,371,695

#### *(4.1.1) Academic Age Groups*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_deciles AS

WITH researcher_deciles AS 
(
  SELECT DISTINCT researcher_ids, incoming_academic_age,
         NTILE(10) OVER(ORDER BY incoming_academic_age) as incoming_age_decile,
  FROM cshdimensionstest.test.flows_1980_2022_age_1
  GROUP BY researcher_ids, incoming_academic_age
), decile_range AS (
    SELECT 
      incoming_age_decile
    , MIN(incoming_academic_age) AS min
    , MAX(incoming_academic_age) AS max
  FROM researcher_deciles
  GROUP BY incoming_age_decile
)
, deciles_joined AS (
  SELECT DISTINCT a.*, CONCAT('Age group id: ', b.incoming_age_decile,' | Range (years): ',  min, '-', max) age_group
  FROM cshdimensionstest.test.flows_1980_2022_age_1 a
  JOIN researcher_deciles b 
    ON a.researcher_ids=b.researcher_ids 
    AND a.incoming_academic_age=b.incoming_academic_age
  JOIN decile_range c 
    ON b.incoming_age_decile=c.incoming_age_decile
)
SELECT * FROM deciles_joined;

--SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_age_deciles; -- 17,371,695

SELECT * FROM cshdimensionstest.test.flows_1980_2022_age_deciles
ORDER BY researcher_ids, p1, p2, unit1, unit1
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,unit1,unit2,p1,p2,outgoing_academic_age,incoming_academic_age,age_group
0,ur.01000000010.53,grid.461843.c,grid.506261.6,2007,2015,9,9,Age group id: 7 | Range (years): 7-9
1,ur.010000000201.99,grid.260542.7,grid.412046.5,2014,2014,1,1,Age group id: 1 | Range (years): 1-2
2,ur.010000000201.99,grid.453140.7,grid.412046.5,2014,2014,1,1,Age group id: 1 | Range (years): 1-2
3,ur.010000000201.99,grid.412046.5,grid.39158.36,2014,2020,7,7,Age group id: 6 | Range (years): 6-7
4,ur.01000000021.08,grid.136593.b,grid.17091.3e,2010,2013,4,4,Age group id: 4 | Range (years): 4-5
5,ur.01000000021.08,grid.17091.3e,grid.469958.f,2013,2016,7,7,Age group id: 6 | Range (years): 6-7
6,ur.01000000021.08,grid.469958.f,grid.10251.37,2016,2021,12,12,Age group id: 9 | Range (years): 12-17
7,ur.01000000021.08,grid.469958.f,grid.440269.d,2016,2021,12,12,Age group id: 9 | Range (years): 12-17
8,ur.010000000667.71,grid.412795.c,grid.419886.a,2020,2022,3,3,Age group id: 3 | Range (years): 3-4
9,ur.010000001234.00,grid.412370.3,grid.414474.6,2020,2021,2,2,Age group id: 2 | Range (years): 2-3


In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*) FROM  cshdimensionstest.test.flows_1980_2022_age_deciles;

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,17371695


We have the following groups
1. Age group id: 1 | Range (years): 1-2
2. Age group id: 2 | Range (years): 2-3
3. Age group id: 3 | Range (years): 3-4
4. Age group id: 4 | Range (years): 4-5
5. Age group id: 5 | Range (years): 5-6
6. Age group id: 6 | Range (years): 6-7
7. Age group id: 7 | Range (years): 7-9
8. Age group id: 8 | Range (years): 9-12
9. Age group id: 9 | Range (years): 12-17
10. Age group id: 10 | Range (years): 17-43


In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_groups AS

WITH outgoing AS (
SELECT unit1 as node, p2 as date_d, 
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 1 | Range (years): 1-2') AS outgoing_age_1_2_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 2 | Range (years): 2-3') AS outgoing_age_2_3_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 3 | Range (years): 3-4') AS outgoing_age_3_4_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 4 | Range (years): 4-5') AS outgoing_age_4_5_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 5 | Range (years): 5-6') AS outgoing_age_5_6_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 6 | Range (years): 6-7') AS outgoing_age_6_7_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 7 | Range (years): 7-9') AS outgoing_age_7_9_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 8 | Range (years): 9-12') AS outgoing_age_9_12_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 9 | Range (years): 12-17') AS outgoing_age_12_17_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 10 | Range (years): 17-43') AS outgoing_age_17_43_y
     FROM cshdimensionstest.test.flows_1980_2022_age_deciles
GROUP BY unit1, p2 )
, incoming AS (
  SELECT unit2 as node, p2 as date_d, 
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 1 | Range (years): 1-2') AS incoming_age_1_2_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 2 | Range (years): 2-3') AS incoming_age_2_3_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 3 | Range (years): 3-4') AS incoming_age_3_4_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 4 | Range (years): 4-5') AS incoming_age_4_5_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 5 | Range (years): 5-6') AS incoming_age_5_6_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 6 | Range (years): 6-7') AS incoming_age_6_7_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 7 | Range (years): 7-9') AS incoming_age_7_9_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 8 | Range (years): 9-12') AS incoming_age_9_12_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 9 | Range (years): 12-17') AS incoming_age_12_17_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 10 | Range (years): 17-43') AS incoming_age_17_43_y
     FROM cshdimensionstest.test.flows_1980_2022_age_deciles
GROUP BY unit2, date_d
) ,
age_aggregated AS 
(
  SELECT a.*, b.incoming_age_1_2_y, b.incoming_age_2_3_y, b.incoming_age_3_4_y,
  b.incoming_age_4_5_y, b.incoming_age_5_6_y, b.incoming_age_6_7_y, b.incoming_age_7_9_y,
  b.incoming_age_9_12_y, b.incoming_age_12_17_y, b.incoming_age_17_43_y
  FROM outgoing a
  LEFT JOIN incoming b 
    ON a.node=b.node 
    AND a.date_d=b.date_d
)
SELECT *
FROM age_aggregated
ORDER BY node, date_d;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_groups
ORDER BY node, date_d
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_age_1_2_y,outgoing_age_2_3_y,outgoing_age_3_4_y,outgoing_age_4_5_y,outgoing_age_5_6_y,outgoing_age_6_7_y,outgoing_age_7_9_y,outgoing_age_9_12_y,...,incoming_age_1_2_y,incoming_age_2_3_y,incoming_age_3_4_y,incoming_age_4_5_y,incoming_age_5_6_y,incoming_age_6_7_y,incoming_age_7_9_y,incoming_age_9_12_y,incoming_age_12_17_y,incoming_age_17_43_y
0,grid.1001.0,1980,5,0,0,0,0,0,0,0,...,4,0,0,0,0,0,0,0,0,0
1,grid.1001.0,1981,3,2,0,0,0,0,0,0,...,4,3,0,0,0,0,0,0,0,0
2,grid.1001.0,1982,7,1,1,0,0,0,0,0,...,4,8,5,0,0,0,0,0,0,0
3,grid.1001.0,1983,3,3,4,5,0,0,0,0,...,5,8,5,4,0,0,0,0,0,0
4,grid.1001.0,1984,4,5,4,8,4,0,0,0,...,2,1,7,4,3,0,0,0,0,0
5,grid.1001.0,1985,2,7,7,5,5,2,0,0,...,11,8,5,10,6,4,0,0,0,0
6,grid.1001.0,1986,6,7,8,9,5,8,0,0,...,4,0,5,3,4,7,1,0,0,0
7,grid.1001.0,1987,6,5,6,7,6,9,7,0,...,17,9,7,9,10,8,3,0,0,0
8,grid.1001.0,1988,9,9,10,13,6,11,19,0,...,11,10,5,9,8,9,8,0,0,0
9,grid.1001.0,1989,7,11,12,8,9,12,16,3,...,4,10,9,7,5,8,8,4,0,0


#### *(4.1.2) Academic Age Statistics*

In [ ]:
# Let's average the academic age at the level of institutions
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_statistics AS

WITH outgoing AS (
  SELECT 
    unit1 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(outgoing_academic_age), 0) AS sum_outgoing_academic_age,
    ROUND(STDDEV(outgoing_academic_age), 3) AS stdev_outgoing_academic_age,
    ROUND(APPROX_QUANTILES(outgoing_academic_age, 3)[OFFSET(1)], 3) AS median_outgoing_academic_age
  FROM cshdimensionstest.test.flows_1980_2022_age_1
  GROUP BY unit1, p2
),
incoming AS (
  SELECT 
    unit2 as node,
    p2 as move_year,
    COUNT(DISTINCT researcher_ids) AS num_researchers,
    IFNULL(SUM(incoming_academic_age), 0) AS sum_incoming_academic_age,
    ROUND(STDDEV(incoming_academic_age), 3) AS stdev_incoming_academic_age,
    ROUND(APPROX_QUANTILES(incoming_academic_age, 3)[OFFSET(1)], 3) AS median_incoming_academic_age
  FROM cshdimensionstest.test.flows_1980_2022_age_1
  GROUP BY unit2, p2
),
age_aggregated AS (
  SELECT 
    a.node,
    a.move_year,
    a.num_researchers as outflows,
    b.num_researchers as inflows,
    ROUND(a.sum_outgoing_academic_age / a.num_researchers, 3) AS mean_outgoing_academic_age,
    ROUND(a.sum_outgoing_academic_age / (a.num_researchers * a.num_researchers), 3) AS mean_normalized_outgoing_academic_age,
    median_outgoing_academic_age,
    stdev_outgoing_academic_age,
    ROUND(b.sum_incoming_academic_age / b.num_researchers, 3) AS mean_incoming_academic_age,
    ROUND(b.sum_incoming_academic_age / (b.num_researchers * b.num_researchers), 3) AS mean_normalized_incoming_academic_age,
    median_incoming_academic_age,
    stdev_incoming_academic_age
  FROM outgoing a
  JOIN incoming b
  ON a.node = b.node AND a.move_year = b.move_year
)
SELECT * 
FROM age_aggregated
ORDER BY node, move_year;

<p>The outgoing_academic_age represents the average academic age of researchers who are leaving the node (i.e., the institution or country) during the specified year. It is calculated by taking the average of the difference between the year of departure (p2) and the minimum academic age of the researcher (min_academic_age) for each unique combination of unit1 (i.e., the node) and p2 (i.e., the year). A lower value for outgoing_academic_age would indicate that researchers leaving the node are generally younger and/or have spent less time in academia, while a higher value would indicate the opposite.</p>
<p>This code also computes the average normalized academic ages of outgoing and incoming researchers by dividing the academic age by the total number of flows. The outgoing_flows and incoming_flows columns are added to the academic_age CTE to capture the total number of flows for each researcher in each year. The mean_normalized_outgoing_academic_age and mean_normalized_incoming_academic_age columns then compute the average normalized academic age by taking the average of the normalized academic age values.</p>

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutional_academic_age_statistics_lagged AS

WITH academic_age AS 
(
  SELECT 
    node,
    move_year,
    mean_incoming_academic_age,
    mean_outgoing_academic_age,
    LAG(mean_incoming_academic_age) OVER (PARTITION BY node ORDER BY move_year) AS prev_mean_incoming_academic_age,
    LAG(mean_outgoing_academic_age) OVER (PARTITION BY node ORDER BY move_year) AS prev_mean_outgoing_academic_age
  FROM cshdimensionstest.test.flows_1980_2022_age_statistics
  )
SELECT
  node,
  move_year,
  ROUND(((mean_incoming_academic_age - prev_mean_incoming_academic_age) / prev_mean_incoming_academic_age) * 100, 3) AS incoming_academic_age_pct_change,
  ROUND(((mean_outgoing_academic_age - prev_mean_outgoing_academic_age) / prev_mean_outgoing_academic_age) * 100, 3) AS outgoing_academic_age_pct_change
FROM academic_age;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_institutional_academic_age_statistics_lagged
ORDER BY node, move_year
LIMIT 50;

A positive value would indicate an increase in the average academic age, while a negative value would indicate a decrease. This measure would be less sensitive to changes in the number of flows than the absolute change in the average academic age.

In [ ]:
# joining the age indicator tables
%%bigquery --project $project_id
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutional_academic_age_stats AS
SELECT a.*, b.incoming_academic_age_pct_change, b.outgoing_academic_age_pct_change
FROM cshdimensionstest.test.flows_1980_2022_age_statistics a
LEFT JOIN cshdimensionstest.test.flows_1980_2022_institutional_academic_age_statistics_lagged b
ON a.node=b.node and a.move_year=b.move_year;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_institutional_academic_age_stats
ORDER BY node, move_year
LIMIT 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,move_year,outflows,inflows,mean_outgoing_academic_age,mean_normalized_outgoing_academic_age,median_outgoing_academic_age,stdev_outgoing_academic_age,mean_incoming_academic_age,mean_normalized_incoming_academic_age,median_incoming_academic_age,stdev_incoming_academic_age,incoming_academic_age_pct_change,outgoing_academic_age_pct_change
0,grid.1001.0,1980,5,4,1.000,0.200,1.0,0.000,1.000,0.250,1.0,0.000,NaN,NaN
1,grid.1001.0,1981,5,7,1.600,0.320,1.0,0.548,1.429,0.204,1.0,0.535,42.900,60.000
2,grid.1001.0,1982,9,17,1.444,0.160,1.0,0.726,2.235,0.131,2.0,0.752,56.403,-9.750
3,grid.1001.0,1983,15,21,3.067,0.204,3.0,1.100,2.524,0.120,2.0,1.008,12.931,112.396
4,grid.1001.0,1984,23,17,3.522,0.153,2.0,1.393,3.353,0.197,3.0,1.272,32.845,14.835
5,grid.1001.0,1985,27,41,3.630,0.134,3.0,1.427,3.463,0.084,2.0,1.737,3.281,3.066
6,grid.1001.0,1986,43,23,3.767,0.088,3.0,1.850,4.478,0.195,3.0,1.989,29.310,3.774
7,grid.1001.0,1987,44,55,4.886,0.111,3.0,2.320,4.109,0.075,2.0,2.212,-8.240,29.705
8,grid.1001.0,1988,72,59,5.250,0.073,3.0,2.540,4.322,0.073,2.0,2.384,5.184,7.450
9,grid.1001.0,1989,77,53,5.143,0.067,3.0,2.622,5.170,0.098,3.0,2.656,19.621,-2.038


In [ ]:
%%bigquery --project $project_id
select count(distinct node) from cshdimensionstest.test.flows_1980_2022_institutional_academic_age_stats -- 46351 institutions

In [ ]:
# lets merge with the age stats
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics AS
select a.*
,mean_outgoing_academic_age
,stdev_outgoing_academic_age
,mean_normalized_outgoing_academic_age
,median_outgoing_academic_age
,outgoing_academic_age_pct_change
,mean_incoming_academic_age
,stdev_incoming_academic_age
,mean_normalized_incoming_academic_age
,median_incoming_academic_age
,incoming_academic_age_pct_change
from cshdimensionstest.test.flows_1980_2022_institutional_total_flows_agg a
left join cshdimensionstest.test.flows_1980_2022_institutional_academic_age_stats b
on a.node=b.node and a.date_d=b.move_year;

select * 
from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
order by node, date_d
limit 50;

Query is running:   0%|          |

Downloading:   0%|          |

,node,date_d,outgoing_flows,percentage_outflows,incoming_flows,percentage_inflows,total_flows,total_net_flows,net_mobility_rate,mean_outgoing_academic_age,stdev_outgoing_academic_age,mean_normalized_outgoing_academic_age,median_outgoing_academic_age,outgoing_academic_age_pct_change,mean_incoming_academic_age,stdev_incoming_academic_age,mean_normalized_incoming_academic_age,median_incoming_academic_age,incoming_academic_age_pct_change
0,grid.1001.0,1980,5,55.6,4,44.4,9,-1,-11.1,1.000,0.000,0.200,1.0,NaN,1.000,0.000,0.250,1.0,NaN
1,grid.1001.0,1981,5,41.7,7,58.3,12,2,16.7,1.600,0.548,0.320,1.0,60.000,1.429,0.535,0.204,1.0,42.900
2,grid.1001.0,1982,9,34.6,17,65.4,26,8,30.8,1.444,0.726,0.160,1.0,-9.750,2.235,0.752,0.131,2.0,56.403
3,grid.1001.0,1983,15,41.7,21,58.3,36,6,16.7,3.067,1.100,0.204,3.0,112.396,2.524,1.008,0.120,2.0,12.931
4,grid.1001.0,1984,23,57.5,17,42.5,40,-6,-15.0,3.522,1.393,0.153,2.0,14.835,3.353,1.272,0.197,3.0,32.845
5,grid.1001.0,1985,27,39.7,41,60.3,68,14,20.6,3.630,1.427,0.134,3.0,3.066,3.463,1.737,0.084,2.0,3.281
6,grid.1001.0,1986,43,65.2,23,34.8,66,-20,-30.3,3.767,1.850,0.088,3.0,3.774,4.478,1.989,0.195,3.0,29.310
7,grid.1001.0,1987,44,44.4,55,55.6,99,11,11.1,4.886,2.320,0.111,3.0,29.705,4.109,2.212,0.075,2.0,-8.240
8,grid.1001.0,1988,72,55.0,59,45.0,131,-13,-9.9,5.250,2.540,0.073,3.0,7.450,4.322,2.384,0.073,2.0,5.184
9,grid.1001.0,1989,77,59.2,53,40.8,130,-24,-18.5,5.143,2.622,0.067,3.0,-2.038,5.170,2.656,0.098,3.0,19.621


<div>
  <h3>How to Interpret the Mean Normalized Outgoing Academic Age:</h3>
  <ul>
    <li>The "Normalized Outgoing Academic Age" is the average age of outgoing researchers, adjusted by the total number of outgoing flows.</li>
    <li>A higher value for the "Normalized Outgoing Academic Age" indicates that, on average, researchers leaving the institution are older, relative to the total number of outgoing flows.</li>
    <li>A lower value for the "Normalized Outgoing Academic Age" indicates that, on average, researchers leaving the institution are younger, relative to the total number of outgoing flows.</li>
  </ul>
</div>


#### *(4.1.3) Age and Distance Statistics*

In [ ]:
%%bigquery --project $project_id 

CREATE OR REPLACE TABLE cshdimensionstest.test.average_distance_source AS

WITH distance AS (
  SELECT 
    a.researcher_ids,
    a.unit1 as source_institution,
    a.unit2 as destination_institution,
    a.p2 as move_year,
    b.latitude as source_latitude,
    b.longitude as source_longitude,
    c.latitude as destination_latitude,
    c.longitude as destination_longitude,
    6371 * 2 * ASIN(SQRT(POWER(SIN(((c.latitude - b.latitude) * ACOS(-1) / 180) / 2), 2) +
      COS((b.latitude * ACOS(-1) / 180)) * COS((c.latitude * ACOS(-1) / 180)) *
      POWER(SIN(((c.longitude - b.longitude) * ACOS(-1) / 180) / 2), 2))) AS distance_km
FROM cshdimensionstest.test.flows_1980_2022 a
JOIN cshdimensionstest.test.organisations b on a.unit1=b.id
JOIN cshdimensionstest.test.organisations c on a.unit2=c.id
)
, distance_summary AS (
SELECT 
  source_institution
, CAST(move_year AS INT) move_year
, AVG(distance_km) as average_distance_travelled_km
, COUNT(distinct researcher_ids) as outflows
FROM distance
GROUP BY source_institution, move_year )
SELECT 
  source_institution,
  move_year,
  average_distance_travelled_km,
  outflows,
  CASE
      WHEN average_distance_travelled_km <  100 Then '< 100 km'
      WHEN average_distance_travelled_km >= 100 AND average_distance_travelled_km < 1000 Then '≥ 100 km < 1000 km'
      WHEN average_distance_travelled_km >= 1000 AND average_distance_travelled_km < 5000 Then '≥ 1000 km < 5000 km'
      WHEN average_distance_travelled_km >= 5000 AND average_distance_travelled_km < 10000 Then '≥ 5000 km < 10000 km'
      WHEN average_distance_travelled_km >= 10000 AND average_distance_travelled_km < 20000 Then '≥ 10000 km < 20000 km'
      ELSE '≥ 20000 km'
  END AS distance_category
FROM distance_summary
ORDER BY source_institution,move_year;

SELECT * 
FROM cshdimensionstest.test.average_distance_source 
ORDER BY source_institution, move_year
LIMIT 200;

Query is running:   0%|          |

Downloading:   0%|          |

,source_institution,move_year,average_distance_travelled_km,outflows,distance_category
0,grid.1001.0,1980,6672.830547,5,≥ 5000 km < 10000 km
1,grid.1001.0,1981,4231.268098,5,≥ 1000 km < 5000 km
2,grid.1001.0,1982,14713.548021,9,≥ 10000 km < 20000 km
3,grid.1001.0,1983,10514.479211,15,≥ 10000 km < 20000 km
4,grid.1001.0,1984,12258.226770,23,≥ 10000 km < 20000 km
...,...,...,...,...,...
195,grid.1004.5,2004,6195.141836,78,≥ 5000 km < 10000 km
196,grid.1004.5,2005,6749.803494,65,≥ 5000 km < 10000 km
197,grid.1004.5,2006,6502.224719,75,≥ 5000 km < 10000 km
198,grid.1004.5,2007,7187.965490,94,≥ 5000 km < 10000 km


In [ ]:
%%bigquery --project $project_id 

CREATE OR REPLACE TABLE cshdimensionstest.test.average_distance_destination AS

WITH distance AS (
  SELECT 
    a.researcher_ids,
    a.unit1 as source_institution,
    a.unit2 as destination_institution,
    a.p2 as move_year,
    b.latitude as source_latitude,
    b.longitude as source_longitude,
    c.latitude as destination_latitude,
    c.longitude as destination_longitude,
    6371 * 2 * ASIN(SQRT(POWER(SIN(((c.latitude - b.latitude) * ACOS(-1) / 180) / 2), 2) +
      COS((b.latitude * ACOS(-1) / 180)) * COS((c.latitude * ACOS(-1) / 180)) *
      POWER(SIN(((c.longitude - b.longitude) * ACOS(-1) / 180) / 2), 2))) AS distance_km
FROM cshdimensionstest.test.flows_1980_2022 a
JOIN cshdimensionstest.test.organisations b on a.unit1=b.id
JOIN cshdimensionstest.test.organisations c on a.unit2=c.id
), distance_summary AS (
SELECT 
  destination_institution
, CAST(move_year AS INT) move_year
, AVG(distance_km) as average_distance_travelled_km
, COUNT(distinct researcher_ids) as inflows
FROM distance
GROUP BY destination_institution, move_year )
SELECT 
  destination_institution,
  move_year,
  average_distance_travelled_km,
  inflows,
  CASE
      WHEN average_distance_travelled_km <  100 Then '< 100 km'
      WHEN average_distance_travelled_km >= 100 AND average_distance_travelled_km < 1000 Then '≥ 100 km < 1000 km'
      WHEN average_distance_travelled_km >= 1000 AND average_distance_travelled_km < 5000 Then '≥ 1000 km < 5000 km'
      WHEN average_distance_travelled_km >= 5000 AND average_distance_travelled_km < 10000 Then '≥ 5000 km < 10000 km'
      WHEN average_distance_travelled_km >= 10000 AND average_distance_travelled_km < 20000 Then '≥ 10000 km < 20000 km'
      ELSE '≥ 20000 km'
  END AS distance_category
FROM distance_summary
ORDER BY destination_institution,move_year;

SELECT * 
FROM cshdimensionstest.test.average_distance_destination 
ORDER BY destination_institution, move_year
LIMIT 100;


Query is running:   0%|          |

Downloading:   0%|          |

,destination_institution,move_year,average_distance_travelled_km,inflows,distance_category
0,grid.1001.0,1980,11483.229331,4,≥ 10000 km < 20000 km
1,grid.1001.0,1981,6390.930344,7,≥ 5000 km < 10000 km
2,grid.1001.0,1982,6694.229502,17,≥ 5000 km < 10000 km
3,grid.1001.0,1983,9948.854060,21,≥ 5000 km < 10000 km
4,grid.1001.0,1984,12108.005065,17,≥ 10000 km < 20000 km
...,...,...,...,...,...
95,grid.10025.36,1989,1662.028915,77,≥ 1000 km < 5000 km
96,grid.10025.36,1990,1694.587623,86,≥ 1000 km < 5000 km
97,grid.10025.36,1991,1630.674284,97,≥ 1000 km < 5000 km
98,grid.10025.36,1992,1576.249300,99,≥ 1000 km < 5000 km


In [ ]:
%%bigquery --project $project_id 

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics_with_distances AS

SELECT 
  a.*,
  b.average_distance_travelled_km AS mean_distance_travelled_to_destination,
  b.distance_category AS distance_category_to_destination,
  c.average_distance_travelled_km AS mean_distance_travelled_from_source,
  c.distance_category AS distance_category_from_source,
  d.total_researchers,
  d.non_mobile_researchers,
  d.pct_non_mobile,
  d.mobile_researchers,
  d.pct_mobile
FROM cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics a
LEFT JOIN cshdimensionstest.test.average_distance_destination b 
  ON a.node = b.destination_institution AND a.date_d = b.move_year
LEFT JOIN cshdimensionstest.test.average_distance_source c 
  ON a.node = c.source_institution AND a.date_d = c.move_year
LEFT JOIN cshdimensionstest.test.au_pub_history_1980_2022_population_statistics d
 ON a.node = d.grid_ids and a.date_d =d.year;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics_with_distances
ORDER BY node, date_d
LIMIT 50;

### ***(4.2) Top k Institutions***

#### *Top Destination Institutions*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations AS
SELECT
  origin_node,
  date_d,
  ARRAY_AGG(CONCAT('Destination: ', destination_node
    , ' | Rank: ', CAST(top_rank AS STRING)
    , ' | Outflows: ', CAST(incoming_flows AS STRING)) 
    ORDER BY top_rank ASC LIMIT 5) AS top_5_destinations
FROM (
  SELECT
    origin_node,
    date_d,
    destination_node,
    incoming_flows,
    ROW_NUMBER() OVER (PARTITION BY origin_node, date_d ORDER BY incoming_flows DESC) AS top_rank
  FROM (
    SELECT
      unit1 AS origin_node,
      unit2 AS destination_node,
      p2 AS date_d,
      COUNT(DISTINCT researcher_ids) AS incoming_flows
    FROM
      cshdimensionstest.test.flows_1980_2022
    GROUP BY
      origin_node, destination_node, date_d
    HAVING incoming_flows > 1
  ) AS flows
) AS ranked_flows
GROUP BY
  origin_node, date_d
ORDER BY
  origin_node, date_d;

In [ ]:
 %%bigquery --project $project_id
 select * from 
 cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations 
 order by origin_node, date_d
 limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,origin_node,date_d,top_5_destinations
0,grid.1001.0,1983,[Destination: grid.1003.2 | Rank: 1 | Outflows...
1,grid.1001.0,1984,[Destination: grid.21100.32 | Rank: 1 | Outflo...
2,grid.1001.0,1985,[Destination: grid.241116.1 | Rank: 1 | Outflo...
3,grid.1001.0,1986,[Destination: grid.1016.6 | Rank: 1 | Outflows...
4,grid.1001.0,1987,[Destination: grid.1013.3 | Rank: 1 | Outflows...
5,grid.1001.0,1988,[Destination: grid.1018.8 | Rank: 1 | Outflows...
6,grid.1001.0,1989,[Destination: grid.5335.0 | Rank: 1 | Outflows...
7,grid.1001.0,1990,[Destination: grid.1013.3 | Rank: 1 | Outflows...
8,grid.1001.0,1991,[Destination: grid.414685.a | Rank: 1 | Outflo...
9,grid.1001.0,1992,[Destination: grid.415306.5 | Rank: 1 | Outflo...


#### *Top Source Institutions*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_top_5_sources AS
SELECT
  destination_node,
  date_d,
  ARRAY_AGG(CONCAT('Source: ', origin_node
    , ' | Rank: ', CAST(top_rank AS STRING)
    , ' | Inflows: ', CAST(incoming_flows AS STRING)) 
    ORDER BY top_rank ASC LIMIT 5) AS top_5_destinations
FROM (
  SELECT
    origin_node,
    date_d,
    destination_node,
    incoming_flows,
    ROW_NUMBER() OVER (PARTITION BY destination_node, date_d ORDER BY incoming_flows DESC) AS top_rank
  FROM (
    SELECT
      unit1 AS origin_node,
      unit2 AS destination_node,
      p2 AS date_d,
      COUNT(DISTINCT researcher_ids) AS incoming_flows
    FROM
      cshdimensionstest.test.flows_1980_2022
    GROUP BY
      origin_node, destination_node, date_d
    HAVING incoming_flows > 1
  ) AS flows
) AS ranked_flows
GROUP BY
  destination_node, date_d
ORDER BY
  destination_node, date_d;

#### *Top Overall Destination - Source Institutions*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_top_5_sources_overall AS
--Top 5 overall source institutions for each destination institution:
WITH source_destination_counts AS (
  SELECT unit1 as source_institution, unit2 as destination_institution, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022 
  GROUP BY source_institution, destination_institution
),

source_ranks AS (
  SELECT source_institution, destination_institution, count_80_22,
    RANK() OVER (PARTITION BY destination_institution ORDER BY count_80_22 DESC) AS source_rank
  FROM source_destination_counts
),

top_sources_per_destination AS (
  SELECT source_institution, destination_institution, count_80_22 AS source_count_1980_2022, source_rank as source_rank_1980_2022
  FROM source_ranks
  WHERE source_rank <= 5 AND count_80_22 > 1
)

SELECT 
  source_institution,
  destination_institution,
  source_count_1980_2022,
  source_rank_1980_2022
FROM top_sources_per_destination
ORDER BY source_rank_1980_2022, source_count_1980_2022, destination_institution ASC;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_institutions_top_5_sources_overall
ORDER by destination_institution, source_rank_1980_2022, source_count_1980_2022
LIMIT 5 ;

Query is running:   0%|          |

Downloading:   0%|          |

,source_institution,destination_institution,source_count_1980_2022,source_rank_1980_2022
0,grid.1013.3,grid.1001.0,238,1
1,grid.1005.4,grid.1001.0,222,2
2,grid.413314.0,grid.1001.0,215,3
3,grid.1008.9,grid.1001.0,210,4
4,grid.1003.2,grid.1001.0,195,5


In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations_overall AS
--Top 5 overall destination institutions for each source institution:
WITH source_destination_counts AS (
  SELECT unit1 as source_institution, unit2 as destination_institution, COUNT(distinct researcher_ids) AS count_80_22
  FROM cshdimensionstest.test.flows_1980_2022 
  GROUP BY source_institution, destination_institution
),

destination_ranks AS (
  SELECT source_institution, destination_institution, count_80_22,
    RANK() OVER (PARTITION BY source_institution ORDER BY count_80_22 DESC) AS destination_rank
  FROM source_destination_counts
),

top_destinations_per_source AS (
  SELECT source_institution, destination_institution, count_80_22 AS destination_count_1980_2022, destination_rank AS destination_rank_1980_2022,
  FROM destination_ranks
  WHERE destination_rank <= 5 AND count_80_22 > 1
)

SELECT 
  source_institution,
  destination_institution,
  destination_count_1980_2022,
  destination_rank_1980_2022
FROM top_destinations_per_source
ORDER BY destination_rank_1980_2022, destination_count_1980_2022, source_institution ASC;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations_overall
ORDER by source_institution, destination_rank_1980_2022, destination_count_1980_2022
LIMIT 5;

Query is running:   0%|          |

Downloading:   0%|          |

,source_institution,destination_institution,destination_count_1980_2022,destination_rank_1980_2022
0,grid.1001.0,grid.1013.3,234,1
1,grid.1001.0,grid.1003.2,222,2
2,grid.1001.0,grid.1005.4,209,3
3,grid.1001.0,grid.1008.9,208,4
4,grid.1001.0,grid.1002.3,162,5


### ***(4.3) Production and Impact***

#### *Author Productivity*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.au_cumulative_pubs_1980_2022 AS

WITH ordered_publications AS (
  SELECT DISTINCT
    researcher_ids,
    year,
    COUNT(DISTINCT pub_id) as n_pubs, 
    ROW_NUMBER() OVER(PARTITION BY researcher_ids ORDER BY year) AS publication_number
  FROM
    cshdimensionstest.test.au_pub_history_1980_2022 
  GROUP BY researcher_ids,  year
)
SELECT
  researcher_ids,
  year,
  SUM(n_pubs) OVER(PARTITION BY researcher_ids ORDER BY year
                                ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
                   ) AS cumulative_publications
FROM
  ordered_publications
ORDER BY
  researcher_ids,
  year;

SELECT * 
FROM  cshdimensionstest.test.au_cumulative_pubs_1980_2022
WHERE researcher_ids = 'ur.01000000010.53'
ORDER BY year;

Query is running:   0%|          |

Downloading:   0%|          |

,researcher_ids,year,cumulative_publications
0,ur.01000000010.53,2007,1
1,ur.01000000010.53,2011,2
2,ur.01000000010.53,2012,3
3,ur.01000000010.53,2013,10
4,ur.01000000010.53,2014,17
5,ur.01000000010.53,2015,34
6,ur.01000000010.53,2016,36
7,ur.01000000010.53,2017,37
8,ur.01000000010.53,2018,40
9,ur.01000000010.53,2019,52


In [13]:
%%bigquery --project $project_id
select count(distinct researcher_ids) from cshdimensionstest.test.au_pub_history_1980_2022

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,6427801


In [8]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_cum_pubs AS

SELECT 
  f.researcher_ids, 
  f.unit1, 
  f.unit2, 
  f.p1,
  f.p2,
  (
    SELECT 
      MAX(a.cumulative_publications)
    FROM 
      cshdimensionstest.test.au_cumulative_pubs_1980_2022 a
    WHERE 
       a.researcher_ids = f.researcher_ids
      AND a.year = f.p2
  ) AS source_cumulative_pubs
FROM 
  cshdimensionstest.test.flows_1980_2022 f;

SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_cum_pubs; -- 	17371695


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,17371695


**Attention: there are authors missing from the pub_history table. Find out why!!**

In [14]:
%%bigquery --project $project_id
select * 
from  cshdimensionstest.test.au_pub_history_1980_2022
where researcher_ids = 'ur.010022351501.26'
--where source_cumulative_pubs = 0

Query is running:   0%|          |

Downloading: |          |

,researcher_ids,pub_id,year,id,grid_ids,grid_id_count


In [ ]:
# calculate the number of incoming authors with at least x publications
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_pub_deciles AS

WITH researcher_deciles AS 
(
  SELECT DISTINCT researcher_ids, source_cumulative_pubs,
         NTILE(10) OVER(ORDER BY source_cumulative_pubs) as source_cumulative_pubs_decile,
  FROM cshdimensionstest.test.flows_1980_2022_cum_pubs
  GROUP BY researcher_ids, source_cumulative_pubs
), decile_range AS (
    SELECT 
      source_cumulative_pubs_decile
    , MIN(source_cumulative_pubs) AS min
    , MAX(source_cumulative_pubs) AS max
  FROM researcher_deciles
  GROUP BY source_cumulative_pubs_decile
)
, deciles_joined AS (
  SELECT DISTINCT a.*, CONCAT('Productivity group id: ', b.source_cumulative_pubs_decile,' | Range (pubs): ',  min, '-', max) pub_group
  FROM cshdimensionstest.test.flows_1980_2022_cum_pubs a
  JOIN researcher_deciles b 
    ON a.researcher_ids=b.researcher_ids 
    AND a.source_cumulative_pubs=b.source_cumulative_pubs
  JOIN decile_range c 
    ON b.source_cumulative_pubs_decile=c.source_cumulative_pubs_decile
)
SELECT * FROM deciles_joined;

--SELECT COUNT(*) FROM cshdimensionstest.test.flows_1980_2022_age_deciles; -- 17,371,695

SELECT * FROM cshdimensionstest.test.flows_1980_2022_pub_deciles
ORDER BY researcher_ids, move_year, unit1, unit1
LIMIT 20;


Executing query with job ID: 37914aa8-a0de-49fb-9c19-8db0d9d14b16
Query executing: 33.09s


ERROR:
 400 Query error: Unrecognized name: move_year at [31:26]

Location: US
Job ID: 37914aa8-a0de-49fb-9c19-8db0d9d14b16



In [ ]:
%%bigquery --project $project_id
SELECT DISTINCT pub_group FROM  cshdimensionstest.test.flows_1980_2022_pub_deciles;

Query is running:   0%|          |

Downloading:   0%|          |

,pub_group
0,Productivity group id: 9 | Range (pubs): 21-42
1,Productivity group id: 10 | Range (pubs): 42-7797
2,Productivity group id: 1 | Range (pubs): 1-2
3,Productivity group id: 2 | Range (pubs): 2-3
4,Productivity group id: 3 | Range (pubs): 3-3
5,Productivity group id: 4 | Range (pubs): 3-5
6,Productivity group id: 5 | Range (pubs): 5-6
7,Productivity group id: 6 | Range (pubs): 6-9
8,Productivity group id: 7 | Range (pubs): 9-13
9,Productivity group id: 8 | Range (pubs): 13-21


# TBC..

#### *Author Productivity Groups*

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2022_age_groups AS

WITH outgoing AS (
SELECT unit1 as node, p2 as date_d, 
       COUNTIF(IFNULL(age_group, '') = 'Productivity group id: 1 | Range (pubs): 0-2') AS outgoing_age_1_2_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 2 | Range (years): 2-3') AS outgoing_age_2_3_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 3 | Range (years): 3-4') AS outgoing_age_3_4_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 4 | Range (years): 4-5') AS outgoing_age_4_5_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 5 | Range (years): 5-6') AS outgoing_age_5_6_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 6 | Range (years): 6-7') AS outgoing_age_6_7_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 7 | Range (years): 7-9') AS outgoing_age_7_9_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 8 | Range (years): 9-12') AS outgoing_age_9_12_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 9 | Range (years): 12-17') AS outgoing_age_12_17_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 10 | Range (years): 17-43') AS outgoing_age_17_43_y
     FROM cshdimensionstest.test.flows_1980_2022_age_deciles
GROUP BY unit1, p2 )
, incoming AS (
  SELECT unit2 as node, p2 as date_d, 
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 1 | Range (years): 1-2') AS incoming_age_1_2_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 2 | Range (years): 2-3') AS incoming_age_2_3_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 3 | Range (years): 3-4') AS incoming_age_3_4_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 4 | Range (years): 4-5') AS incoming_age_4_5_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 5 | Range (years): 5-6') AS incoming_age_5_6_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 6 | Range (years): 6-7') AS incoming_age_6_7_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 7 | Range (years): 7-9') AS incoming_age_7_9_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 8 | Range (years): 9-12') AS incoming_age_9_12_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 9 | Range (years): 12-17') AS incoming_age_12_17_y,
       COUNTIF(IFNULL(age_group, '') = 'Age group id: 10 | Range (years): 17-43') AS incoming_age_17_43_y
     FROM cshdimensionstest.test.flows_1980_2022_age_deciles
GROUP BY unit2, date_d
) ,
age_aggregated AS 
(
  SELECT a.*, b.incoming_age_1_2_y, b.incoming_age_2_3_y, b.incoming_age_3_4_y,
  b.incoming_age_4_5_y, b.incoming_age_5_6_y, b.incoming_age_6_7_y, b.incoming_age_7_9_y,
  b.incoming_age_9_12_y, b.incoming_age_12_17_y, b.incoming_age_17_43_y
  FROM outgoing a
  LEFT JOIN incoming b 
    ON a.node=b.node 
    AND a.date_d=b.date_d
)
SELECT *
FROM age_aggregated
ORDER BY node, date_d;

SELECT *
FROM cshdimensionstest.test.flows_1980_2022_age_groups
ORDER BY node, date_d
LIMIT 50;

# TBC

#### *Author Impact*

#### *Author Gender*

# REVISION

## ***Retention Rates***

To calculate the Scientists' retention rate per calendar year, I follow these steps:

    1. Identify the number of scientists affiliated in each institution in each year.
    2. Identify the number of scientists who were affiliated in the same institution in the previous year.
    3. Calculate the retention rate as the number of scientists who remained in the same institution divided by the total number of scientists enrolled in that institution.
    4. Repeat the process for each year and each institution.

This procedure calculates the retention rate of scientists affiliated with an institution from one year to the next. The retention rate is defined as *the ratio of the number of scientists affiliated with an institution in a given year to the number of scientists affiliated with the same institution in the previous year*.

The interpretation of the retention rate:

    A value of 1 means that the institution was able to retain the same number of scientists as the previous year
    , while a value greater than 1 indicates an increase in the number of affiliated scientists
    , and a value less than 1 indicates a decrease. 
    The output can be used to assess the ability of institutions
    to retain their affiliated scientists over time

This code calculates the retention rate of scientists for each institution and year, which is defined as the proportion of scientists who are affiliated with the institution in the current year and were also affiliated with the same institution in the previous year.

Advantages of this indicator include:

    It provides a clear picture of how well institutions are retaining their existing scientists.
    It is a simple indicator that can be easily understood.

Limitations of this indicator include:

    It does not take into account new scientists who may have joined the institution in the current year, so it may not provide a complete picture of the institution's ability to attract scientists.
    It assumes that the number of scientists affiliated with an institution in any given year is a good proxy for the institution's ability to attract and retain scientists. This may not always be the case, as factors such as funding, working conditions, and location can also play a significant role in attracting and retaining scientists.
    It also does not account for scientists who have left the institution but have not left the field entirely, which may lead to an over-estimation of the institution's retention rate.

The formula for the retention rate is calculated as:

retention_rate = affiliated_scientists_prev_year / affiliated_scientists

where:

    affiliated_scientists_prev_year represents the number of scientists affiliated with the institution in the previous year
    affiliated_scientists represents the number of scientists affiliated with the institution in the current year.

In [ ]:
%%bigquery --project $project_id
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_retention AS
WITH scientist_retention 
AS (
SELECT
research_orgs,
year,
researcher_ids,
LAG(research_orgs) OVER (PARTITION BY researcher_ids ORDER BY year) AS prev_research_orgs,
LAG(year) OVER (PARTITION BY researcher_ids ORDER BY year) AS prev_year,
FROM cshdimensionstest.test.researchers_after_1980_simplified
WHERE year >= 1980 AND year <= 2000
),
retention_rate AS (
SELECT
research_orgs,
year,
COUNT(DISTINCT researcher_ids) AS remaining_scientists
FROM scientist_retention
WHERE research_orgs = prev_research_orgs AND year = prev_year + 2 -- checks how many scientists remained affiliated in the previous 2 years
GROUP BY research_orgs, year
),
retention_rate_with_affiliation AS (
SELECT
retention_rate.research_orgs,
retention_rate.year,
retention_rate.remaining_scientists,
scientist_affiliation.affiliated_scientists,
CASE 
  WHEN scientist_affiliation.affiliated_scientists = 0 THEN 0
  ELSE retention_rate.remaining_scientists / scientist_affiliation.affiliated_scientists
END AS retention_rate
FROM retention_rate
JOIN (
SELECT
research_orgs,
year,
COUNT(DISTINCT researcher_ids) AS affiliated_scientists
FROM cshdimensionstest.test.researchers_after_1980_simplified
WHERE year >= 1980 AND year <= 2000
GROUP BY research_orgs, year
) AS scientist_affiliation
ON retention_rate.research_orgs = scientist_affiliation.research_orgs 
AND retention_rate.year = scientist_affiliation.year)
SELECT 
research_orgs,
year,
affiliated_scientists,
remaining_scientists,
retention_rate
FROM retention_rate_with_affiliation;

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project $project_id
SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_retention 
ORDER BY  research_orgs, year
LIMIT 30;

Query is running:   0%|          |

Downloading:   0%|          |

,research_orgs,year,affiliated_scientists,remaining_scientists,retention_rate
0,grid.1001.0,1981,252,42,0.166667
1,grid.1001.0,1982,339,60,0.176991
2,grid.1001.0,1983,392,71,0.181122
3,grid.1001.0,1984,512,81,0.158203
4,grid.1001.0,1985,548,99,0.180657
5,grid.1001.0,1986,597,103,0.172529
6,grid.1001.0,1987,760,106,0.139474
7,grid.1001.0,1988,841,155,0.184304
8,grid.1001.0,1989,880,149,0.169318
9,grid.1001.0,1990,970,137,0.141237


In [ ]:
%%bigquery --project $project_id
# merge the table with the main indicator table
CREATE OR REPLACE TABLE cshdimensionstest.test.flows_1980_2000_institutional_total_indicators AS
SELECT a.*, b.affiliated_scientists, b.remaining_scientists, retention_rate
FROM cshdimensionstest.test.flows_1980_2000_institutional_total_flow_indicators a
LEFT JOIN cshdimensionstest.test.flows_1980_2000_institutional_retention b 
ON a.research_orgs=b.research_orgs 
and a.year=b.year;

DROP TABLE IF EXISTS cshdimensionstest.test.flows_1980_2000_institutional_total_flow_indicators;
DROP TABLE IF EXISTS cshdimensionstest.test.flows_1980_2000_institutional_retention;

SELECT * 
FROM cshdimensionstest.test.flows_1980_2000_institutional_total_indicators
ORDER BY research_orgs, year asc
LIMIT 30;

Query is running:   0%|          |

Downloading:   0%|          |

,year,research_orgs,population_year,inflows,outflows,net_migration_rate,total_migrants,non_migrant_percentage,affiliated_scientists,affiliated_scientists_prev_year,retention_rate
0,1980,grid.1001.0,202,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,NaN
1,1981,grid.1001.0,252,12,25,-5.15873,37,85.317460,252,202,0.801587
2,1982,grid.1001.0,339,46,40,1.76991,86,74.631268,339,252,0.743363
3,1983,grid.1001.0,392,75,76,-0.25510,151,61.479592,392,339,0.864796
4,1984,grid.1001.0,512,106,138,-6.25000,244,52.343750,512,392,0.765625
5,1985,grid.1001.0,548,146,160,-2.55474,306,44.160584,548,512,0.934307
6,1986,grid.1001.0,597,166,184,-3.01508,350,41.373534,597,548,0.917923
7,1987,grid.1001.0,760,253,212,5.39474,465,38.815789,760,597,0.785526
8,1988,grid.1001.0,841,278,314,-4.28062,592,29.607610,841,760,0.903686
9,1989,grid.1001.0,880,350,335,1.70455,685,22.159091,880,841,0.955682


* One thing that we notice is that the non-migrant percentage is negative.

* The reason why this happens is that we calculate the population number for each year and do not take into account the population that stayed from the previous year. We need to make some adjustments.

* To count only the distinct cumulative population for each year, you could add a column to your query that identifies whether a scientist is still affiliated with the same institution in the next year. Then, you can sum up the number of scientists who are still affiliated with the same institution and divide that by the total number of affiliated scientists to get the cumulative non-migrant percentage. 

* non-mobile = what is the percentage of non-mobile researchers who have never left up until that point in time

-- we need to change the `non_migrant_percentage `indicator and align it with the `retention_rate` indicators


# Visualizations (TEST)

In [ ]:
client = bigquery.Client()

# Make the query
df = pd.io.gbq.read_gbq('''
select * 
from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
where node in (
  select node
  from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
  group by node
  having sum(total_flows) > 100000
  order by sum(total_flows) desc)
''', project_id=project_id, dialect='standard')

# Average outgoing 
# Create a plotly scatter plot
fig = go.Figure()

for node in df['node'].unique():
    node_df = df[df['node'] == node]
    fig.add_trace(go.Scatter(x=node_df['date_d']
                             , y=node_df['incoming_academic_age_pct_change'] 
                             , mode='lines+markers'
                             , name=node
                             , marker=dict(color=node_df['total_flows'], showscale=False, colorscale='Blues', opacity=0.5)
                             , hovertemplate='Year: %{x}<br>' + 'Outgoing Academic Age: %{y}<br>' + 'Total Flows: %{marker.color:.2f}<br>' + 'Node: ' + node + '<br><extra></extra>'))

# Add a shared color axis
fig.update_layout(
    coloraxis=dict(
        colorbar=dict(
            title="Total Flows",
            title_font=dict(size=18),
            tickfont=dict(size=14),
            len=0.5,
            tickangle=-45,
            tickmode='array',
            tickvals=[0, 500, 1000, 5000, 10000, 50000]
        ),
        showscale=True,
    ),
    title="Average Academic Age by Year and Institution",
    xaxis_title="Year",
    yaxis_title="Average Normalized Outgoing Academic Age",
    showlegend=True,
)

# Show the plot
pio.show(fig)


# save the plot in plotly graph studio and edit it there
#!pip install chart-studio
#import chart_studio.plotly as py
#py.sign_in(username='Ferreir4', api_key='40YuWFKy73EGkgEjddBA')
# save the plot as an HTML file in your Google Drive folder
#url = py.plot(fig, filename='my_plot', auto_open=False)

In [ ]:
client = bigquery.Client()

# Make the query
df = pd.io.gbq.read_gbq('''
select * 
from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
where node in (
  select node
  from cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics
  group by node
  having sum(total_flows) > 50000
  order by sum(total_flows) desc)
''', project_id=project_id, dialect='standard')

# Average outgoing 
# Create a plotly scatter plot
fig = go.Figure()

for node in df['node'].unique():
    node_df = df[df['node'] == node]
    fig.add_trace(go.Scatter(x=node_df['date_d']
                             , y=node_df['percentage_outflows'] 
                             , mode='lines+markers'
                             , name=node
                             , marker=dict(color=node_df['total_flows'], showscale=False, colorscale='Blues', opacity=0.5)
                             , hovertemplate='Year: %{x}<br>' + 'Incoming Academic Age: %{y}<br>' + 'Total Flows: %{marker.color:.2f}<br>' + 'Node: ' + node + '<br><extra></extra>'))

# Add a shared color axis
fig.update_layout(
    coloraxis=dict(
        colorbar=dict(
            title="Total Flows",
            title_font=dict(size=18),
            tickfont=dict(size=14),
            len=0.5,
            tickangle=-45,
            tickmode='array',
            tickvals=[0, 500, 1000, 5000, 10000, 50000]
        ),
        showscale=True,
    ),
    title="Average Academic Age by Year and Institution",
    xaxis_title="Year",
    yaxis_title="Average Normalized Incoming Academic Age",
    showlegend=True,
)

# Show the plot
pio.show(fig)

# save the plot in plotly graph studio and edit it there
#!pip install chart-studio
#import chart_studio.plotly as py
#py.sign_in(username='Ferreir4', api_key='40YuWFKy73EGkgEjddBA')
# save the plot as an HTML file in your Google Drive folder
#url = py.plot(fig, filename='my_plot', auto_open=False)

# ***Data Exports***

#### Organisations' Metadata

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT id, name,  status, established, types, wikipedia_url, address.latitude, address.longitude, address.city, address.country, address.country_code 
FROM `dimensions-ai.data_analytics.grid` 
ORDER BY id, established
"""
organisations_metadata = client.query(sql).to_dataframe()
organisations_metadata.head(10)

# save the dataset
organisations_metadata.to_csv('organisations_metadata.csv', index_label='row_no', encoding = 'utf-8-sig')
#files.download('organisations_metadata.csv')

#from google.colab import drive
#drive.mount('/content/drive')
!cp organisations_metadata.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

#### Organisations' Edges

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_institutional_flows` 
ORDER BY geoid_o, geoid_d, date_d
"""
organisations_edges = client.query(sql).to_dataframe()
organisations_edges.head(10)

organisations_edges.to_csv('organisations_edges.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp organisations_edges.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

#### Organisations' Indicators

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_institutions_flow_statistics_with_distances` 
ORDER BY node, date_d
"""
organisations_indicators_1 = client.query(sql).to_dataframe()
organisations_indicators_1.head(10)

organisations_indicators_1.to_csv('organisations_indicators_1.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp organisations_indicators_1.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_age_groups` 
ORDER BY node, date_d
"""
organisations_indicators_age_group_counts = client.query(sql).to_dataframe()
organisations_indicators_age_group_counts.head(10)

organisations_indicators_age_group_counts.to_csv('organisations_indicators_age_group_counts.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp organisations_indicators_age_group_counts.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_institutions_top_5_sources_overall` 
ORDER BY source_institution
"""
organisations_indicators_top_5_sources = client.query(sql).to_dataframe()
organisations_indicators_top_5_sources.head(10)

organisations_indicators_top_5_sources.to_csv('organisations_indicators_top_5_sources.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp organisations_indicators_top_5_sources.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project=project_id)

sql = """
SELECT *
FROM `cshdimensionstest.test.flows_1980_2022_institutions_top_5_destinations_overall` 
ORDER BY destination_institution
"""
organisations_indicators_top_5_destinations= client.query(sql).to_dataframe()
organisations_indicators_top_5_destinations.head(10)

organisations_indicators_top_5_destinations.to_csv('organisations_indicators_top_5_destinations.csv', index_label='row_no', encoding = 'utf-8-sig')

!cp organisations_indicators_top_5_destinations.csv "/content/drive/My Drive/CSH-DIMENSIONS PROJECT/BigQuery-results"

# PART III - Coverage

1. **Make a hello world program**
1. **Connect resources to each other:**
 e.g., can I print the GBQ data in a website (print=show any table) for instance?
1. **Other considerations**
* how to run queries fast enough (users should not have delays)
* how does the interface look like
* how to put all calculations in one query?
* how to connect the web interface to google bigquery?
* what if multiple users use it? performance?
